# Import Packages

In [1]:
# we will import everything here
import numpy as np

from src.read_file import *
from src.parser import *

%reload_ext autoreload 
%autoreload 2 

# Modeling Environment Setup

In [2]:
# read the initial state of the cellular automata from a file
X = read_file("input.txt")
print("Input state matrix: ")
print(X)

# parse the input and get the required parameters
rows, cols, pedestrians, obstacles, targets = parser(X)

print("number of rows and columns")
print(rows, cols)

print("coordinate list of the pedestrians")
print(pedestrians)

print("coordinate list of the obstacles")
print(obstacles)

print("coordinate list of the targets")
print(targets)

Input state matrix: 
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 1 2 3 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
number of rows and columns
5 5
coordinate list of the pedestrians
[[2 1]]
coordinate list of the obstacles
[[2 2]]
coordinate list of the targets
[[2 3]]
